In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sklearn.preprocessing import StandardScaler

In [4]:
 df = pd.read_excel("IWA.xlsx", sheet_name='Final Raw Sample(3%)')
 df.head()

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 6,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed
0,2019,SAGA PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Multi-line Insurance,Activities auxiliary to financial intermediati...,-0.010415,-0.046989,-1.148703e+07,-1.123710e+07,-2602.469978,...,-61541.905533,-1059.042606,-4.601425,-0.370166,-1295.252263,-5.484575,71.252219,71.252219,-468.060022,0.006135
1,2019,BURSA MALAYSIA BHD,MALAYSIA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-0.006172,-0.012754,-7.244098e+05,-7.079386e+05,-165.923420,...,-4823.472569,-168.146428,-1.369721,-0.435001,-80.693593,-1.632612,16.655875,16.655875,-28.698267,0.043215
2,2019,INTERTEK GROUP PLC,UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN I...,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.005523,-0.034259,-2.188151e+07,-2.140410e+07,-5012.270759,...,-117104.605466,-3804.051175,-16.528190,-1.329625,-2484.645598,-19.700439,255.935962,255.935962,-892.152710,0.011467
3,2019,JSE LIMITED,SOUTH AFRICA,Financial Exchanges & Data,Activities auxiliary to financial intermediati...,-0.005327,NaN,-8.341370e+05,-8.153911e+05,-184.180256,...,-4396.445439,-901.413470,-0.184194,-0.363841,-91.327105,-0.219547,0.618135,0.618135,-33.382235,0.016390
4,2019,BUREAU VERITAS SA,FRANCE,Research & Consulting Services,Activities auxiliary to financial intermediati...,-0.002534,-0.018462,-1.448471e+07,-1.416778e+07,-3433.217789,...,-77276.296027,-4116.308632,-36.717300,-3.292484,-1668.361350,-43.764435,588.677794,588.677794,-591.706523,0.033005


In [5]:
semiconductors_df = df[df['GICS Sub-Industry'] == 'Semiconductors']
semiconductors_df.head()

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 6,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed
544,2019,VANGUARD INTL SEMICONDUCTOR,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.094108,-0.394019,-8.898888e+07,-8.097028e+07,-18678.724831,...,-6.668843e+06,-3020.588532,-36.986879,-8.139551,-9283.526296,-44.085754,538.238182,538.238182,-3362.808405,0.007411
545,2019,ON SEMICONDUCTOR CORP,UNITED STATES OF AMERICA,Semiconductors,Manufacture of electrical machinery and appara...,-0.085773,-0.735144,-4.732855e+08,-3.064513e+08,-70849.885295,...,-1.608762e+08,-891029.127734,-278.986785,-67.715965,-34994.297985,-332.532595,3803.157912,3803.157912,-12631.619461,0.169190
551,2019,WALTON ADVANCED ENG INC,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.057381,NaN,-1.290486e+07,-1.168842e+07,-2709.982026,...,-1.021560e+06,-718.396916,-8.796716,-1.936153,-1341.710874,-10.485066,128.011030,128.011030,-484.366412,0.013895
552,2019,UNITED MICROELECTRONICS CORP,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.057289,NaN,-2.838305e+08,-2.605703e+08,-59963.443221,...,-1.892041e+07,-15826.035274,-86.482702,-42.646154,-29815.744233,-103.081289,1234.179287,1234.179287,-10807.191160,0.004732
553,2019,DB HITEK CO LTD,"KOREA, REPUBLIC OF",Semiconductors,Manufacture of electrical machinery and appara...,-0.055385,-0.252383,-3.872616e+07,-3.548926e+07,-8173.447291,...,-2.642865e+06,-2540.077897,-11.666579,-1.377889,-4070.895839,-13.905740,167.495588,167.495588,-1474.998155,0.029233


In [6]:
# Group the data by "Company Name" and count unique years for each group
company_years_count = semiconductors_df.groupby("Company Name")["Year"].nunique()

# Filter companies with at least 10 years of data
filtered_companies = company_years_count[company_years_count >= 10].index

# Create a new DataFrame with only the companies that meet the criteria
filtered_df = semiconductors_df[semiconductors_df["Company Name"].isin(filtered_companies)]
filtered_df.head()

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 6,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed
544,2019,VANGUARD INTL SEMICONDUCTOR,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.094108,-0.394019,-8.898888e+07,-8.097028e+07,-18678.724831,...,-6.668843e+06,-3.020589e+03,-36.986879,-8.139551,-9283.526296,-44.085754,538.238182,538.238182,-3362.808405,0.007411
552,2019,UNITED MICROELECTRONICS CORP,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.057289,NaN,-2.838305e+08,-2.605703e+08,-59963.443221,...,-1.892041e+07,-1.582604e+04,-86.482702,-42.646154,-29815.744233,-103.081289,1234.179287,1234.179287,-10807.191160,0.004732
561,2019,MACRONIX INTL CO LTD,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.043421,-0.501489,-5.079780e+07,-4.739269e+07,-10800.844989,...,-2.615827e+06,-3.737062e+03,-7.257502,-10.070296,-5376.756675,-8.650431,96.882670,96.882670,-1961.390871,0.005121
578,2019,FIRST SOLAR INC,UNITED STATES OF AMERICA,Semiconductors,Manufacture of electrical machinery and appara...,-0.024568,-0.369486,-7.525364e+07,-5.288874e+07,-12521.570676,...,-2.101377e+07,-4.946314e+05,-154.872173,-37.590736,-6021.724284,-184.596721,2111.223048,2111.223048,-2132.875485,0.053054
580,2019,TEXAS INSTRUMENTS INC,UNITED STATES OF AMERICA,Semiconductors,Manufacture of electrical machinery and appara...,-0.023956,-0.060588,-3.445640e+08,-2.344083e+08,-52707.560222,...,-1.040595e+08,-2.322563e+06,-80.166544,-176.489116,-25994.488330,-95.552874,350.786933,350.786933,-9456.331315,0.024635


In [7]:
with_rev = filtered_df.copy()
with_rev["Revenue"] = with_rev["Total Environmental Cost"]/with_rev["Environmental Intensity (Sales)"]
with_rev

,Year,Company Name,Country,GICS Sub-Industry,Industry (Exiobase),Environmental Intensity (Sales),Environmental Intensity (Op Inc),Total Environmental Cost,Working Capacity,Fish Production Capacity,...,SDG 12.2,SDG 14.1,SDG 14.2,SDG 14.3,SDG 14.c,SDG 15.1,SDG 15.2,SDG 15.5,% Imputed,Revenue
544,2019,VANGUARD INTL SEMICONDUCTOR,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.094108,-0.394019,-8.898888e+07,-8.097028e+07,-18678.724831,...,-3.020589e+03,-36.986879,-8.139551,-9283.526296,-44.085754,538.238182,538.238182,-3362.808405,0.007411,9.456030e+08
552,2019,UNITED MICROELECTRONICS CORP,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.057289,NaN,-2.838305e+08,-2.605703e+08,-59963.443221,...,-1.582604e+04,-86.482702,-42.646154,-29815.744233,-103.081289,1234.179287,1234.179287,-10807.191160,0.004732,4.954381e+09
561,2019,MACRONIX INTL CO LTD,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.043421,-0.501489,-5.079780e+07,-4.739269e+07,-10800.844989,...,-3.737062e+03,-7.257502,-10.070296,-5376.756675,-8.650431,96.882670,96.882670,-1961.390871,0.005121,1.169897e+09
578,2019,FIRST SOLAR INC,UNITED STATES OF AMERICA,Semiconductors,Manufacture of electrical machinery and appara...,-0.024568,-0.369486,-7.525364e+07,-5.288874e+07,-12521.570676,...,-4.946314e+05,-154.872173,-37.590736,-6021.724284,-184.596721,2111.223048,2111.223048,-2132.875485,0.053054,3.063117e+09
580,2019,TEXAS INSTRUMENTS INC,UNITED STATES OF AMERICA,Semiconductors,Manufacture of electrical machinery and appara...,-0.023956,-0.060588,-3.445640e+08,-2.344083e+08,-52707.560222,...,-2.322563e+06,-80.166544,-176.489116,-25994.488330,-95.552874,350.786933,350.786933,-9456.331315,0.024635,1.438300e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13396,2010,STMICROELECTRONICS NV,NETHERLANDS,Semiconductors,Manufacture of electrical machinery and appara...,-0.020918,-0.390642,-1.639477e+08,-1.548458e+08,-35672.427604,...,-1.675387e+03,-34.712655,-0.988545,-17793.786686,-41.375040,513.079771,513.079771,-6441.507507,0.002649,7.837716e+09
13400,2010,POWERTECH TECHNOLOGY INC,"TAIWAN, PROVINCE OF CHINA",Semiconductors,Manufacture of electrical machinery and appara...,-0.019819,-0.089040,-2.568837e+07,-2.513462e+07,-5954.173075,...,-4.004698e+03,-49.037231,-10.793070,-2903.056499,-58.448925,713.596572,713.596572,-1034.316740,0.036247,1.296179e+09
13401,2010,TEXAS INSTRUMENTS INC,UNITED STATES OF AMERICA,Semiconductors,Manufacture of electrical machinery and appara...,-0.019735,-0.062599,-2.756252e+08,-2.268199e+08,-51129.997151,...,-2.150978e+06,-92.717874,-163.450377,-25210.460349,-110.513175,597.894077,597.894077,-9163.129768,0.025526,1.396600e+10
13405,2010,FIRST SOLAR INC,UNITED STATES OF AMERICA,Semiconductors,Manufacture of electrical machinery and appara...,-0.015312,-0.053072,-3.885349e+07,-3.109243e+07,-7447.493860,...,-3.908075e+05,-122.364275,-29.700385,-3534.865144,-145.849596,1668.074212,1668.074212,-1240.047894,0.074137,2.537459e+09


In [8]:
X = (with_rev.loc[:,"Working Capacity":"Wood Production Capacity"])
y = (with_rev.loc[:,"Revenue"])
X

,Working Capacity,Fish Production Capacity,Crop Production Capacity,Meat Production Capacity,Biodiversity,Abiotic Resources,Water production capacity (Drinking water & Irrigation Water),Wood Production Capacity
544,-8.097028e+07,-18678.724831,-1.074192e+06,-2.515435e+05,-3400.254246,-3.020589e+03,-6.668843e+06,1076.476364
552,-2.605703e+08,-59963.443221,-3.448132e+06,-8.074392e+05,-10894.747005,-1.582604e+04,-1.892041e+07,2468.358573
561,-4.739269e+07,-10800.844989,-6.263392e+05,-1.466387e+05,-1968.738429,-3.737062e+03,-2.615827e+06,193.765339
578,-5.288874e+07,-12521.570676,-6.871356e+05,-1.587742e+05,-2289.661413,-4.946314e+05,-2.101377e+07,4222.446095
580,-2.344083e+08,-52707.560222,-3.008403e+06,-7.035948e+05,-9537.454989,-2.322563e+06,-1.040595e+08,701.573866
...,...,...,...,...,...,...,...,...
13396,-1.548458e+08,-35672.427604,-2.062960e+06,-4.819689e+05,-6476.650903,-1.675387e+03,-6.514127e+06,1026.159541
13400,-2.513462e+07,-5954.173075,-3.319142e+05,-7.693653e+04,-1083.962462,-4.004698e+03,-1.352842e+05,1427.193143
13401,-2.268199e+08,-51129.997151,-2.915937e+06,-6.819887e+05,-9256.963298,-2.150978e+06,-4.299717e+07,1195.788154
13405,-3.109243e+07,-7447.493860,-4.006665e+05,-9.209986e+04,-1363.924225,-3.908075e+05,-6.872009e+06,3336.148424


In [9]:
scaler = StandardScaler()
#X_standardized = scaler.fit_transform(X)
y = np.log(y)

In [10]:
X = sm.add_constant(X)

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                Revenue   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.877
Method:                 Least Squares   F-statistic:                     89.14
Date:                Thu, 28 Sep 2023   Prob (F-statistic):           1.10e-39
Time:                        11:55:28   Log-Likelihood:                -55.388
No. Observations:                 100   AIC:                             128.8
Df Residuals:                      91   BIC:                             152.2
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                                                    coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------